# TESTING CORRELATIONS BETWEEN MICROBIOME AND RESISTOME
We want to see if there's any direct correlation between any particular zOTU and any particular ARG and MGE. To begin with, I'll try just with taxa that make up each core, to deal with way less data at once 

In [1]:
import os

import pandas as pd
from scipy import stats as stat
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import numpy as np

from tqdm.notebook import tqdm

In [2]:
script_dir = os.getcwd()
os.chdir("../data/r_data")
data_dir = os.getcwd()
os.chdir("../../results/Corr_res")
res_dir = os.getcwd()

In [3]:
os.chdir(data_dir)
micro_df = pd.read_csv("rclr_counts_resistome_format.csv")
micro_df.rename(columns = {"Unnamed: 0": "ZOTU"}, inplace = True) # this works because the unnamed column gets automatically assigned that name 
micro_df

,ZOTU,argl_1,argl_16,argl_18,argl_22,argl_2,argl_23,argl_24,argl_25,argl_26,...,argl_10,argl_11,argl_12,argl_4,argl_5,argl_6,argl_13,argl_14,argl_17,argl_15
0,zOTU_1038,0.000000,0.000000,0.000000,-0.562286,0.000000,-0.812174,-1.057733,-1.305141,0.000000,...,0.000000,-0.526270,2.054452,0.00000,0.000000,0.000000,0.0,-1.0,0.0,0.0
1,zOTU_255,0.000000,-0.103321,0.000000,0.000000,-2.687746,0.000000,0.000000,0.000000,0.000000,...,-0.972064,1.334483,-0.205574,0.00000,-2.893764,-1.802858,2.0,2.0,-1.0,-1.0
2,zOTU_885,0.381853,1.374781,-0.222600,-1.168421,0.808762,-2.891616,-0.605748,-1.815967,-1.589599,...,-1.195208,0.000000,0.000000,0.00000,0.000000,0.000000,1.0,0.0,-1.0,1.0
3,zOTU_638,0.000000,0.000000,0.000000,0.407115,0.000000,0.574120,0.292194,1.998076,1.067157,...,-1.482890,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
4,zOTU_1166,-1.322895,-1.281976,-2.707506,0.000000,-1.994599,0.000000,0.000000,1.442130,-0.203305,...,-2.581502,0.000000,-0.744570,0.00000,0.000000,0.000000,-1.0,0.0,-2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2032,zOTU_1599,0.000000,0.000000,0.000000,0.889967,0.000000,-0.119027,-0.295593,-1.815967,-1.589599,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
2033,zOTU_1336,0.000000,0.000000,0.000000,0.130862,0.000000,-0.183565,0.215233,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
2034,zOTU_910,0.000000,0.000000,0.000000,1.749349,0.000000,1.713555,1.539651,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
2035,zOTU_575,1.465198,1.864329,-0.142557,0.000000,1.574934,0.000000,0.000000,0.000000,0.000000,...,-0.502060,-0.680421,-1.081043,0.00000,0.000000,0.000000,1.0,-2.0,0.0,1.0


In [4]:
arg_df = pd.read_csv("count_arg.csv")
mge_df = pd.read_csv("count_mge.csv")
arg_df.rename(columns = {"Assay": "ARG"}, inplace = True)
mge_df.rename(columns = {"Assay": "MGE"}, inplace = True)
arg_df.head()

,ARG,argl_25,argl_27,argl_20,argl_22,argl_26,argl_23,argl_24,argl_19,argl_21,...,argl_12,argl_16,argl_4,argl_5,argl_6,argl_1,argl_2,argl_3,argl_11,argl_14
0,aacC2,0.000222,0.000512,0.000056,0.000211,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,aacA/aphD,0.000046,0.000039,0.000000,0.000149,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,aac(6')-II,0.006918,0.006194,0.000000,0.006457,0.004699,0.007621,0.008630,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,aphA3,0.000000,0.002323,0.000000,0.000818,0.001205,0.000647,0.000738,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,sat4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# Checking if colnames are the same
arg_colnames = arg_df.columns.values.tolist()
micro_colnames = micro_df.columns.values.tolist()
mge_colnames = mge_df.columns.values.tolist()
print(sorted(mge_colnames))
print(sorted(arg_colnames))
print(sorted(micro_colnames))

['MGE', 'argl_1', 'argl_10', 'argl_11', 'argl_12', 'argl_13', 'argl_14', 'argl_15', 'argl_16', 'argl_17', 'argl_18', 'argl_19', 'argl_2', 'argl_20', 'argl_21', 'argl_22', 'argl_23', 'argl_24', 'argl_25', 'argl_26', 'argl_27', 'argl_3', 'argl_4', 'argl_5', 'argl_6', 'argl_7', 'argl_8', 'argl_9']
['ARG', 'argl_1', 'argl_10', 'argl_11', 'argl_12', 'argl_13', 'argl_14', 'argl_15', 'argl_16', 'argl_17', 'argl_18', 'argl_19', 'argl_2', 'argl_20', 'argl_21', 'argl_22', 'argl_23', 'argl_24', 'argl_25', 'argl_26', 'argl_27', 'argl_3', 'argl_4', 'argl_5', 'argl_6', 'argl_7', 'argl_8', 'argl_9']
['ZOTU', 'argl_1', 'argl_10', 'argl_11', 'argl_12', 'argl_13', 'argl_14', 'argl_15', 'argl_16', 'argl_17', 'argl_18', 'argl_19', 'argl_2', 'argl_20', 'argl_21', 'argl_22', 'argl_23', 'argl_24', 'argl_25', 'argl_26', 'argl_27', 'argl_3', 'argl_4', 'argl_5', 'argl_6', 'argl_7', 'argl_8', 'argl_9']


Now, it doesn't make sense to check for correlations as is, so I'm going to study plastic and soil correlations by separate

In [6]:
plastic_micro = micro_df[["ZOTU", "argl_7", "argl_8", "argl_9", "argl_10", "argl_11", "argl_12", "argl_15", "argl_16", "argl_17", 
                          "argl_18", "argl_22", "argl_23", "argl_24", "argl_25", "argl_26", "argl_27"]]
plastic_arg = arg_df[["ARG", "argl_7", "argl_8", "argl_9", "argl_10", "argl_11", "argl_12", "argl_15", "argl_16", "argl_17", 
                          "argl_18", "argl_22", "argl_23", "argl_24", "argl_25", "argl_26", "argl_27"]]
plastic_mge = mge_df[["MGE", "argl_7", "argl_8", "argl_9", "argl_10", "argl_11", "argl_12", "argl_15", "argl_16", "argl_17", 
                          "argl_18", "argl_22", "argl_23", "argl_24", "argl_25", "argl_26", "argl_27"]]
# I can always just re-read them when I need them
del arg_df
del micro_df
del mge_df

In [7]:
# Finally, some cleanup
plastic_micro = plastic_micro.set_index("ZOTU")
plastic_arg = plastic_arg.set_index("ARG")
plastic_mge = plastic_mge.set_index("MGE")
plastic_mge.tail()

,argl_7,argl_8,argl_9,argl_10,argl_11,argl_12,argl_15,argl_16,argl_17,argl_18,argl_22,argl_23,argl_24,argl_25,argl_26,argl_27
MGE,,,,,,,,,,,,,,,,
tnpAd,0.0,0.0,0.0,0.0,0.002742,0.0,0.0,0.0,0.0,0.0,0.000000,0.000665,0.000780,0.000000,0.000000,0.000000
tnpAe,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000246,0.000000,0.000000,0.000000,0.000058,0.000000
tnpAf,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000255,0.000140,0.000169,0.000193,0.000076,0.000140
tnpAg,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.004539,0.004898,0.005035,0.002275,0.002761,0.002183
trfa,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.001057,0.000733,0.000914,0.001365,0.000780,0.006067


The first thing I'm going to do is pre-filter the dataframes. One of the things to consider when calculating correlations is what constitutes enough data to believe the correlation. For example, I have 17 observations for plastic data. If I had 16 0's on both datasets on the same sampling points and 1 value each, I would obtain a perfect correlation but I have no way of knowing if they are truly correlated (as 0 can be both an absence or a value under the detection threshold). So, I will prefilter each dataframe to only keep those variables (rows) with >= 4 observations != 0 (then I'll keep filtering), and I will do it now before joining dataframes for performance reasons

In [8]:
def filter_out_lows(df, min_num):
    """
    Get a count dataframe such as those used in metagenomics and return a version of it where variables with less than min_num observations != 0 are dropped
    - df: the dataframe we want to filter
    - min_num: the minim number of observations we want to be present
    Returns:
    A filtered df
    """
    enough_data_id = [] # Will be used to filter at the last step
    total_obs = df.shape[1] #faster to calculate once at this step
    t_df =  df.T # pandas operations work on columns, so we need to transpose the df
    var_values = t_df.columns.values.tolist()
    
    for var in var_values:
        too_low = t_df[var].value_counts()
        if (0.0 in too_low.index) and ((total_obs - too_low[0]) < min_num): continue
        enough_data_id.append(var)
    return df[df.index.isin(enough_data_id)]

In [9]:
print(plastic_micro.shape)
plastic_micro = filter_out_lows(plastic_micro, 4)
print(plastic_micro.shape)
print("########################")
print(plastic_arg.shape)
plastic_arg = filter_out_lows(plastic_arg, 4)
print(plastic_arg.shape)
print("########################")
print(plastic_mge.shape)
plastic_mge = filter_out_lows(plastic_mge, 4)
print(plastic_mge.shape)

(2037, 16)
(1508, 16)
########################
(310, 16)
(235, 16)
########################
(70, 16)
(55, 16)


And now we should be able to proceed with the left merge. But first, we need to go from the pivot format we have now to long format

In [10]:
def unstack_and_join(df1, df2):
    """
    Highly hardcoded function to make more readable the process of turning two dfs into long format, left-merging them and renaming the resulting columns for readability sake
    - df1: first df to merge, in wide format
    - df2: second df to merge, in wide format aswell
    Returns:
    A long format df with general column names, product of left joining df1 and df2 on the shared sample column
    """
    long_df = (
        df1.set_index(df1.columns[0]) #should always be the column with the var names 
        .stack()
        .reset_index()
        .merge(df2.set_index(df2.columns[0]).stack().reset_index(),
               how = "left", on = "level_1") #level_1 is the generic name that pandas gives to the old index from df1 and df2 after resetting it. As samples are paired, it works
    )
    # the following line works due to how merge orders the final product when specifying "on": keeps the original 3 columns (3 because we resetted the index of the 2 resulting from the stack)
    # and adds the corresponding columns to end of the column list (it adds them "to the right" of the existing ones). So we know the order they will come in 
    long_df.columns = ["var1", "sample", "var1_vals", "var2", "var2_vals"] #this is just for convenience's sake
    return long_df

In [11]:
long_amp = unstack_and_join(plastic_arg, plastic_mge)
long_zap = unstack_and_join(plastic_micro, plastic_arg)
long_zmp = unstack_and_join(plastic_micro, plastic_mge)

long_amp

,var1,sample,var1_vals,var2,var2_vals
0,0.0,argl_8,0.000000,0.0,0.000000
1,0.0,argl_8,0.000000,0.0,0.000000
2,0.0,argl_8,0.000000,0.0,0.000000
3,0.0,argl_8,0.000000,0.0,0.000000
4,0.0,argl_8,0.000000,0.0,0.000000
...,...,...,...,...,...
193870,0.0,argl_27,0.053827,0.0,0.004603
193871,0.0,argl_27,0.053827,0.0,0.309030
193872,0.0,argl_27,0.053827,0.0,0.000140
193873,0.0,argl_27,0.053827,0.0,0.002183


At this point, there's one last filter that I want to apply: drop the var1-var2 pairs in which there aren't more than 6 paired observations different from 0 - 0. E.g., 0 - 1 is valid, and so is 1 - 0, but no 0 - 0.

In [12]:
def filter_out_lows2(df, min_good_pairs):
    """
    The idea now is to get a dataframe product of a merge of two others, with paired observations for two variables, and filter out the variables values pairs that lack more than min_good_pairs != 0.
    - df: the merged df with paired observations for two variables. Column names MUST be ["var1", "sample", "var1_vals", "var2", "var2_vals"], with values in the "*_vals" columns and variable names
    in the "vals*" columns
    - min_good_pairs: the minimum amount of non zero pairs we want
    Returns:
    A dataframe in the format of df1, with "bad" pairings pruned
    """
    good_pairs_lst = []
    for name, group in tqdm(df.groupby(["var1", "var2"])):
        sums = (group["var1_vals"] + group["var2_vals"]).value_counts()
        if (0.0 in sums.index) and (sums[0.0] < min_good_pairs): continue
        good_pairs_lst.append(name)
    return df[df.set_index(["var1", "var2"]).index.isin(good_pairs_lst)]

In [13]:
long_amp = filter_out_lows2(long_amp, min_good_pairs = 7)
long_zap = filter_out_lows2(long_zap, min_good_pairs = 7)
long_zmp = filter_out_lows2(long_zmp, min_good_pairs = 7)

long_zmp

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/2136 [00:00<?, ?it/s]

  0%|          | 0/890 [00:00<?, ?it/s]

,var1,sample,var1_vals,var2,var2_vals
0,0.565059,argl_8,0.034866,0.0,0.000000
1,0.565059,argl_8,0.034866,0.0,0.000000
2,0.565059,argl_8,0.034866,0.0,0.000000
3,0.565059,argl_8,0.034866,0.0,0.000000
4,0.565059,argl_8,0.034866,0.0,0.000000
...,...,...,...,...,...
1244095,0.000000,argl_27,0.000000,0.0,0.004603
1244096,0.000000,argl_27,0.000000,0.0,0.309030
1244097,0.000000,argl_27,0.000000,0.0,0.000140
1244098,0.000000,argl_27,0.000000,0.0,0.002183


Finally, we can calculate the correlations. This whole thing was problematic because I want to permute the statistic to obtain exact p-values, and that destroys the computing time (on the scale of > 10 hours to finish for the smallest df we have), so we need to parallelize it

In [14]:
from multiprocessing import Pool
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) #this will otherwise clutter the screen when calculating corrs

In [15]:
# this is the task I want to parallelize
def permuted_spearman(item):
    # I need each processor to have access to the functions to use, otherwise it will result in a mutex
    import pandas as pd
    import scipy.stats as stat
    
    def statistic(x, y):  # explore all possible pairings by permuting `x`
        rs = stat.spearmanr(x, y).statistic  # ignore pvalue
        dof = len(x) - 2 # will only work for cases where x and y are equal
        transformed = rs * np.sqrt(dof / ((rs+1.0)*(1.0-rs)))
        return transformed 
    group = item[1]
    corr = stat.permutation_test((group["var1_vals"], group["var2_vals"]), statistic, n_resamples = 4999, alternative="two-sided", permutation_type="pairings")
    return [item[0][0], item[0][1], corr.statistic, corr.pvalue]

In [17]:
### AMP RESULTS ####
corr_res = []
with Pool() as pool:
    items = [(name, group) for name, group in long_amp.groupby(["var1", "var2"])]
    print("There's ", len(items), " groups to calculate")
    for result in tqdm(pool.imap(permuted_spearman, items)):
        corr_res.append(result)
corr_res = pd.DataFrame(corr_res, columns = ["var1", "var2", "statistic", "p-val"])
corr_res = corr_res.loc[corr_res["p-val"] <= 0.5]
for name, group in corr_res.groupby(["var1", "var2"]):
    num_data = long_amp.loc[(long_amp["var1"] == name[0]) & (long_amp["var2"] == name[1])]
    coef = stat.spearmanr(num_data["var1_vals"], num_data["var2_vals"]).statistic
    corr_res.loc[(corr_res["var1"] == name[0]) & (corr_res["var2"] == name[1]), "coef"] = coef
corr_res.to_csv("AMP_corrs.csv")

del corr_res
del long_amp

0it [00:00, ?it/s]

ZAP is so big that it bricks my computer when trying to run it. So, I will splice it up into chunks and run one after the other

In [16]:
items = [(name, group) for name, group in long_zap.groupby(["var1", "var2"])]
print(len(items)/5)
items = items[:635]
print(len(items))
#items

254.0
635


In [ ]:
### ZAP RESULTS ####
corr_res = []
with Pool() as pool:
    items = [(name, group) for name, group in long_zap.groupby(["var1", "var2"])]
    items = items[:254]
    print("There's ", len(items), " groups to calculate")
    for result in tqdm(pool.imap(permuted_spearman, items)):
        corr_res.append(result)
corr_res = pd.DataFrame(corr_res, columns = ["var1", "var2", "statistic", "p-val"])
corr_res = corr_res.loc[corr_res["p-val"] <= 0.5]
for name, group in corr_res.groupby(["var1", "var2"]):
    num_data = long_zap.loc[(long_zap["var1"] == name[0]) & (long_zap["var2"] == name[1])]
    coef = stat.spearmanr(num_data["var1_vals"], num_data["var2_vals"]).statistic
    corr_res.loc[(corr_res["var1"] == name[0]) & (corr_res["var2"] == name[1]), "coef"] = coef
corr_res.to_csv("ZAP_corrs_1.csv")

del corr_res
del long_zap

There's  254  groups to calculate


0it [00:00, ?it/s]

In [21]:
### ZMP RESULTS ####
corr_res = []
with Pool() as pool:
    items = [(name, group) for name, group in long_zmp.groupby(["var1", "var2"])]
    print("There's ", len(items), " groups to calculate")
    for result in tqdm(pool.imap(permuted_spearman, items)):
        corr_res.append(result)
corr_res = pd.DataFrame(corr_res, columns = ["var1", "var2", "statistic", "p-val"])
corr_res = corr_res.loc[corr_res["p-val"] <= 0.5]
for name, group in tqdm(corr_res.groupby(["var1", "var2"])):
    num_data = long_zmp.loc[(long_zmp["var1"] == name[0]) & (long_zmp["var2"] == name[1])]
    coef = stat.spearmanr(num_data["var1_vals"], num_data["var2_vals"]).statistic
    corr_res.loc[(corr_res["var1"] == name[0]) & (corr_res["var2"] == name[1]), "coef"] = coef
corr_res.to_csv("ZMP_corrs.csv")

del corr_res
del long_zmp

There's  27257  groups to calculate


0it [00:00, ?it/s]

NameError: name 'long_zap' is not defined

And that should be all plastic results. Let's begin anew with general control results

In [24]:
del plastic_arg
del plastic_mge
del plastic_micro

In [8]:
# Just reusing code
os.chdir(data_dir)
micro_df = pd.read_csv("rclr_counts_resistome_format.csv")
micro_df.rename(columns = {"Unnamed: 0": "ZOTU"}, inplace = True)
arg_df = pd.read_csv("count_arg.csv")
mge_df = pd.read_csv("count_mge.csv")
arg_df.rename(columns = {"Assay": "ARG"}, inplace = True)
mge_df.rename(columns = {"Assay": "MGE"}, inplace = True)

In [9]:
control_micro = micro_df[["ZOTU", "argl_1", "argl_2", "argl_3", "argl_4", "argl_5", "argl_6", "argl_13", "argl_14", "argl_19", 
                          "argl_20", "argl_21"]]
control_arg = arg_df[["ARG", "argl_1", "argl_2", "argl_3", "argl_4", "argl_5", "argl_6", "argl_13", "argl_14", "argl_19", 
                          "argl_20", "argl_21"]]
control_mge = mge_df[["MGE", "argl_1", "argl_2", "argl_3", "argl_4", "argl_5", "argl_6", "argl_13", "argl_14", "argl_19", 
                          "argl_20", "argl_21"]]

del micro_df
del arg_df
del mge_df

In [10]:
warnings.filterwarnings("ignore", category=FutureWarning)

In [11]:
print(control_micro.shape)
control_micro = filter_out_lows(control_micro, 3)
print(control_micro.shape)
print("########################")
print(control_arg.shape)
control_arg = filter_out_lows(control_arg, 3)
print(control_arg.shape)
print("########################")
print(control_mge.shape)
control_mge = filter_out_lows(control_mge, 3)
print(control_mge.shape)

(2037, 12)
(1684, 12)
########################
(310, 12)
(142, 12)
########################
(70, 12)
(32, 12)


In [12]:
long_amc = unstack_and_join(control_arg, control_mge)
long_zac = unstack_and_join(control_micro, control_arg)
long_zmc = unstack_and_join(control_micro, control_mge)

In [13]:
long_amc = filter_out_lows2(long_amc, min_good_pairs = 5)
long_zac = filter_out_lows2(long_zac, min_good_pairs = 5)
long_zmc = filter_out_lows2(long_zmc, min_good_pairs = 5)

print("ZMC has in total ", long_zmc.shape[0]/11, " correlations to calculate")
print("AMC has in total ", long_amc.shape[0]/11, " correlations to calculate")
print("ZAC has in total ", long_zac.shape[0]/11, " correlations to calculate")

  0%|          | 0/4544 [00:00<?, ?it/s]

  0%|          | 0/239128 [00:00<?, ?it/s]

  0%|          | 0/53888 [00:00<?, ?it/s]

ZMC has in total  33163.0  correlations to calculate
AMC has in total  4167.0  correlations to calculate
ZAC has in total  155178.0  correlations to calculate


In [37]:
### AMC RESULTS ####
os.chdir(res_dir)
corr_res = []
with Pool() as pool:
    items = [(name, group) for name, group in long_amc.groupby(["var1", "var2"])]
    print("There's ", len(items), " groups to calculate")
    for result in tqdm(pool.imap(permuted_spearman, items)):
        corr_res.append(result)
corr_res = pd.DataFrame(corr_res, columns = ["var1", "var2", "statistic", "p-val"])
corr_res = corr_res.loc[corr_res["p-val"] <= 0.5]
for name, group in tqdm(corr_res.groupby(["var1", "var2"])):
    num_data = long_amc.loc[(long_amc["var1"] == name[0]) & (long_amc["var2"] == name[1])]
    coef = stat.spearmanr(num_data["var1_vals"], num_data["var2_vals"]).statistic
    corr_res.loc[(corr_res["var1"] == name[0]) & (corr_res["var2"] == name[1]), "coef"] = coef
corr_res.to_csv("AMC_corrs.csv")

del corr_res
del long_amc

There's  4167  groups to calculate


0it [00:00, ?it/s]

  0%|          | 0/3642 [00:00<?, ?it/s]

NameError: name 'long_amp' is not defined

In [14]:
### ZMC RESULTS ####
corr_res = []
with Pool() as pool:
    items = [(name, group) for name, group in long_zmc.groupby(["var1", "var2"])]
    print("There's ", len(items), " groups to calculate")
    for result in tqdm(pool.imap(permuted_spearman, items)):
        corr_res.append(result)
corr_res = pd.DataFrame(corr_res, columns = ["var1", "var2", "statistic", "p-val"])
corr_res = corr_res.loc[corr_res["p-val"] <= 0.5]
for name, group in tqdm(corr_res.groupby(["var1", "var2"])):
    num_data = long_zmc.loc[(long_zmc["var1"] == name[0]) & (long_zmc["var2"] == name[1])]
    coef = stat.spearmanr(num_data["var1_vals"], num_data["var2_vals"]).statistic
    corr_res.loc[(corr_res["var1"] == name[0]) & (corr_res["var2"] == name[1]), "coef"] = coef
corr_res.to_csv("ZMC_corrs.csv")

del corr_res
del long_zmc 

There's  33163  groups to calculate


0it [00:00, ?it/s]

  0%|          | 0/14089 [00:00<?, ?it/s]

In [15]:
### ZAC RESULTS ####
corr_res = []
with Pool() as pool:
    items = [(name, group) for name, group in long_zac.groupby(["var1", "var2"])]
    print("There's ", len(items), " groups to calculate")
    for result in tqdm(pool.imap(permuted_spearman, items)):
        corr_res.append(result)
corr_res = pd.DataFrame(corr_res, columns = ["var1", "var2", "statistic", "p-val"])
corr_res = corr_res.loc[corr_res["p-val"] <= 0.5]
for name, group in tqdm(corr_res.groupby(["var1", "var2"])):
    num_data = long_zac.loc[(long_zac["var1"] == name[0]) & (long_zac["var2"] == name[1])]
    coef = stat.spearmanr(num_data["var1_vals"], num_data["var2_vals"]).statistic
    corr_res.loc[(corr_res["var1"] == name[0]) & (corr_res["var2"] == name[1]), "coef"] = coef
corr_res.to_csv("ZAC_corrs.csv")

del corr_res
del long_zac

There's  155178  groups to calculate


0it [00:00, ?it/s]

  0%|          | 0/64679 [00:00<?, ?it/s]

In [16]:
del control_micro
del control_arg
del control_mge

In [17]:
# Just reusing code
os.chdir(data_dir)
micro_df = pd.read_csv("rclr_counts_resistome_format.csv")
micro_df.rename(columns = {"Unnamed: 0": "ZOTU"}, inplace = True)
arg_df = pd.read_csv("count_arg.csv")
mge_df = pd.read_csv("count_mge.csv")
arg_df.rename(columns = {"Assay": "ARG"}, inplace = True)
mge_df.rename(columns = {"Assay": "MGE"}, inplace = True)

In [18]:
soil_micro = micro_df[["ZOTU", "argl_1", "argl_2", "argl_3", "argl_13", "argl_19", "argl_20", "argl_21"]]
soil_arg = arg_df[["ARG", "argl_1", "argl_2", "argl_3", "argl_13", "argl_19", "argl_20", "argl_21"]]
soil_mge = mge_df[["MGE", "argl_1", "argl_2", "argl_3", "argl_13", "argl_19", "argl_20", "argl_21"]]

del micro_df
del arg_df
del mge_df

In [19]:
warnings.filterwarnings("ignore", category=FutureWarning) #this will otherwise clutter the screen when calculating corrs

In [20]:
print(soil_micro.shape)
soil_micro = filter_out_lows(soil_micro, 3)
print(soil_micro.shape)
print("########################")
print(soil_arg.shape)
soil_arg = filter_out_lows(soil_arg, 3)
print(soil_arg.shape)
print("########################")
print(soil_mge.shape)
soil_mge = filter_out_lows(soil_mge, 3)
print(soil_mge.shape)

(2037, 8)
(1561, 8)
########################
(310, 8)
(139, 8)
########################
(70, 8)
(32, 8)


In [21]:
long_ams = unstack_and_join(soil_arg, soil_mge)
long_zas = unstack_and_join(soil_micro, soil_arg)
long_zms = unstack_and_join(soil_micro, soil_mge)

In [22]:
long_ams = filter_out_lows2(long_ams, min_good_pairs = 5)
long_zas = filter_out_lows2(long_zas, min_good_pairs = 5)
long_zms = filter_out_lows2(long_zms, min_good_pairs = 5)

  0%|          | 0/4448 [00:00<?, ?it/s]

  0%|          | 0/216979 [00:00<?, ?it/s]

  0%|          | 0/49952 [00:00<?, ?it/s]

In [23]:
### AMS RESULTS ####
os.chdir(res_dir)
corr_res = []
with Pool() as pool:
    items = [(name, group) for name, group in long_ams.groupby(["var1", "var2"])]
    print("There's ", len(items), " groups to calculate")
    for result in tqdm(pool.imap(permuted_spearman, items)):
        corr_res.append(result)
corr_res = pd.DataFrame(corr_res, columns = ["var1", "var2", "statistic", "p-val"])
corr_res = corr_res.loc[corr_res["p-val"] <= 0.5]
for name, group in tqdm(corr_res.groupby(["var1", "var2"])):
    num_data = long_ams.loc[(long_ams["var1"] == name[0]) & (long_ams["var2"] == name[1])]
    coef = stat.spearmanr(num_data["var1_vals"], num_data["var2_vals"]).statistic
    corr_res.loc[(corr_res["var1"] == name[0]) & (corr_res["var2"] == name[1]), "coef"] = coef
corr_res.to_csv("AMS_corrs.csv")

del corr_res
del long_ams

There's  443  groups to calculate


0it [00:00, ?it/s]

  0%|          | 0/437 [00:00<?, ?it/s]

In [24]:
### ZMS RESULTS ####
corr_res = []
with Pool() as pool:
    items = [(name, group) for name, group in long_zms.groupby(["var1", "var2"])]
    print("There's ", len(items), " groups to calculate")
    for result in tqdm(pool.imap(permuted_spearman, items)):
        corr_res.append(result)
corr_res = pd.DataFrame(corr_res, columns = ["var1", "var2", "statistic", "p-val"])
corr_res = corr_res.loc[corr_res["p-val"] <= 0.5]
for name, group in tqdm(corr_res.groupby(["var1", "var2"])):
    num_data = long_zms.loc[(long_zms["var1"] == name[0]) & (long_zms["var2"] == name[1])]
    coef = stat.spearmanr(num_data["var1_vals"], num_data["var2_vals"]).statistic
    corr_res.loc[(corr_res["var1"] == name[0]) & (corr_res["var2"] == name[1]), "coef"] = coef
corr_res.to_csv("ZMS_corrs.csv")

del corr_res
del long_zms 

There's  11839  groups to calculate


0it [00:00, ?it/s]

  0%|          | 0/9779 [00:00<?, ?it/s]

In [ ]:
### ZAS RESULTS ####
corr_res = []
with Pool() as pool:
    items = [(name, group) for name, group in long_zas.groupby(["var1", "var2"])]
    print("There's ", len(items), " groups to calculate")
    for result in tqdm(pool.imap(permuted_spearman, items)):
        corr_res.append(result)
corr_res = pd.DataFrame(corr_res, columns = ["var1", "var2", "statistic", "p-val"])
corr_res = corr_res.loc[corr_res["p-val"] <= 0.5]
for name, group in tqdm(corr_res.groupby(["var1", "var2"])):
    num_data = long_zas.loc[(long_zas["var1"] == name[0]) & (long_zas["var2"] == name[1])]
    coef = stat.spearmanr(num_data["var1_vals"], num_data["var2_vals"]).statistic
    corr_res.loc[(corr_res["var1"] == name[0]) & (corr_res["var2"] == name[1]), "coef"] = coef
corr_res.to_csv("ZAS_corrs.csv")

del corr_res
del long_zas

There's  52186  groups to calculate


0it [00:00, ?it/s]

  0%|          | 0/42790 [00:00<?, ?it/s]